# Upload Data to Hub Contracts

In [1]:
import json

from cyber_sdk.client.lcd import LCDClient
from cyberutils.contract import execute_contract

from config import LCD_CLIENT


WALLET_ADDRESS = 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t'

MEMO = 'add entry'

HUB_CONTRACTS = {
    'protocols': 'bostrom1latjl5jde9x7ndu0c9qr4ceyzvyr0ngs95fydqhx8xa6t0md7h4sryuzz0',
    'networks': 'bostrom1ka63rny987cfu79jpfaytn4rjepz4q8na4gtgdz83r79p0yy84gqy6kc8k',
    'channels': 'bostrom1jsc2h2t2r4vpytz0ashmvpt5pqnyqculjq9t9vwy2p5wcu64xytqmzcdvj',
    'tokens': 'bostrom1969quzze7w7a79gy42cnus5t67sgmu6t2yw059arfyn8fzs3hpzs7zjfkx',
    'skills': 'bostrom1ejm29m279e8x4ltf3ksqm443fsdql9rmtuyth6vssmxl72af5yzqrt2mku',
    'contracts': 'bostrom1cfa46qr4ca4n75eys534fzmdtgce9zscvlzfq8fgt4sk6gc250hssks43s'}

UPLOAD_HUB = {
    'protocols': True,
    'networks': True,
    'channels': True,
    'tokens': True,
    'skills': True,
    'contracts': True
}

REQUIRED_KEYS = {
    'protocols': ['data_type'],
    'networks': ['chain_id', 'genesis_hash', 'logo', 'name', 'prefix', 'protocol', 'unbonding_period'],
    'channels': ['active', 'destination_chain_id', 'destination_channel_id', 'explorer_url', 'source_chain_id', 'source_channel_id'],
    'tokens': ['chain_id', 'channel', 'contract', 'decimals', 'logo', 'ticker'],
    'skills': ['endpoint', 'network', 'neuron', 'protocol'],
    'contracts': ['address', 'chain_id', 'execute_cid', 'query_cid', 'version']
}

UPLOAD_DATA_FILES = {k: f'data/{k}.csv' for k in HUB_CONTRACTS.keys()}
TX_FILES = {k: f'txs/tx_{k}_upload_hub_unsigned.json' for k in HUB_CONTRACTS.keys()}

FEE_DENOM = 'boot'

## Create Entries

In [2]:
def hub_upload(
        tx_unsigned_file_name: str,
        items_for_upload: list[dict],
        contract_address: str,
        required_keys: list[str],
        sender: str = WALLET_ADDRESS,
        lcd_client: LCDClient = LCD_CLIENT,
        fee_denom: str = 'boot',
        gas: int = 5_000_000,
        memo='add entry') -> dict:
    _execute_msgs = []
    for _item_for_upload in items_for_upload:
        assert all(_item in _item_for_upload.keys() for _item in required_keys)
        _execute_msg = {'CreateEntry': _item_for_upload}
        _execute_msgs.append(_execute_msg)
    _tx_unsigned = execute_contract(
        execute_msgs=_execute_msgs,
        contract_address=contract_address,
        sender=sender,
        lcd_client=lcd_client,
        fee_denom=fee_denom,
        gas=gas,
        sign_and_broadcast_tx=False,
        memo=memo
    )
    with open(tx_unsigned_file_name, 'w') as _tx_unsigned_file:
        _tx_unsigned_file.write(json.dumps(_tx_unsigned.to_data()))
    return _tx_unsigned.to_data()

### Data fields
- protocols: 'data_type', 'particle'
- networks: 'chain_id', 'genesis_hash', 'logo', 'name', 'prefix', 'protocol', 'unbonding_period', 'particle'
- channels: 'active', 'destination_chain_id', 'destination_channel_id', 'explorer_url', 'source_chain_id', 'source_channel_id', 'particle'
- tokens: 'chain_id', 'channel', 'contract', 'decimals', 'logo', 'ticker', 'particle'
- skills: 'endpoint', 'network', 'neuron', 'protocol', 'particle'
- contracts: 'address', 'chain_id', 'execute_cid', 'query_cid', 'version', 'particle'

`particle` field is optional

In [3]:
tx_unsigned_data = {}
for contract_name in HUB_CONTRACTS.keys():
    if UPLOAD_HUB[contract_name] and contract_name in ('protocols', 'networks', 'channels', 'tokens'):
        with open(f'data/{contract_name}.json', 'r') as file:
            data_for_upload = json.load(file)
        tx_unsigned_data[contract_name] = hub_upload(
            tx_unsigned_file_name=TX_FILES[contract_name],
            items_for_upload=data_for_upload,
            contract_address=HUB_CONTRACTS[contract_name],
            required_keys=REQUIRED_KEYS[contract_name],
            gas=20_000_000
        )

In [4]:
tx_unsigned_data['protocols']

{'body': {'messages': [{'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1latjl5jde9x7ndu0c9qr4ceyzvyr0ngs95fydqhx8xa6t0md7h4sryuzz0',
     'execute_msg': {'CreateEntry': {'data_type': 'ipfs-gateway'}},
     'coins': []}},
   {'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1latjl5jde9x7ndu0c9qr4ceyzvyr0ngs95fydqhx8xa6t0md7h4sryuzz0',
     'execute_msg': {'CreateEntry': {'data_type': 'cyber-rpc'}},
     'coins': []}},
   {'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1latjl5jde9x7ndu0c9qr4ceyzvyr0ngs95fydqhx8xa6t0md7h4sryuzz0',
     'execute_msg': {'CreateEntry': {'data_type': 'cyber-ws'}},
     'coins': []}},
   {'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom

In [5]:
tx_unsigned_data['networks']

{'body': {'messages': [{'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1ka63rny987cfu79jpfaytn4rjepz4q8na4gtgdz83r79p0yy84gqy6kc8k',
     'execute_msg': {'CreateEntry': {'chain_id': 'bostrom',
       'genesis_hash': 'QmYubyVNfghD4xCrTFj26zBwrF9s5GJhi1TmxvrwmJCipr',
       'logo': 'QmSVz41kA5aXttxT1rAXnuJecfazij3wh95X8ewd7Kdc2F',
       'name': 'bostrom',
       'prefix': 'bostrom',
       'protocol': 'cyber-rpc',
       'unbonding_period': '691200'}},
     'coins': []}},
   {'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1ka63rny987cfu79jpfaytn4rjepz4q8na4gtgdz83r79p0yy84gqy6kc8k',
     'execute_msg': {'CreateEntry': {'chain_id': 'space-pussy',
       'genesis_hash': 'QmTR1k469ZNJdmhT2WY67A6fccpj1pRkHTmnZZtYpPDnH7',
       'logo': 'QmSdVMzpY9nuHZXZpvZdFbhhMf3nBGAe4JZ7e2Xdh6LMLr',
       'n

In [6]:
tx_unsigned_data['channels']

{'body': {'messages': [{'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1jsc2h2t2r4vpytz0ashmvpt5pqnyqculjq9t9vwy2p5wcu64xytqmzcdvj',
     'execute_msg': {'CreateEntry': {'active': 'false',
       'destination_chain_id': 'osmosis-1',
       'destination_channel_id': 'channel-79',
       'source_chain_id': 'bostrom',
       'source_channel_id': 'channel-0',
       'explorer_url': 'https://mintscan.io/osmosis'}},
     'coins': []}},
   {'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1jsc2h2t2r4vpytz0ashmvpt5pqnyqculjq9t9vwy2p5wcu64xytqmzcdvj',
     'execute_msg': {'CreateEntry': {'active': 'false',
       'destination_chain_id': 'cosmoshub-4',
       'destination_channel_id': 'channel-225',
       'source_chain_id': 'bostrom',
       'source_channel_id': 'channel-1',
       'explorer_url': 'h

In [7]:
tx_unsigned_data['tokens']

{'body': {'messages': [{'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1969quzze7w7a79gy42cnus5t67sgmu6t2yw059arfyn8fzs3hpzs7zjfkx',
     'execute_msg': {'CreateEntry': {'chain_id': 'bostrom',
       'channel': '',
       'decimals': 0,
       'logo': 'QmaFs8AjTfNy6XFq8xb6boN1eC4nrznaPnvytJtq3FHnkf',
       'ticker': 'boot',
       'contract': 'boot'}},
     'coins': []}},
   {'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'value': {'sender': 'bostrom1d8754xqa9245pctlfcyv8eah468neqzn3a0y0t',
     'contract': 'bostrom1969quzze7w7a79gy42cnus5t67sgmu6t2yw059arfyn8fzs3hpzs7zjfkx',
     'execute_msg': {'CreateEntry': {'chain_id': 'bostrom',
       'channel': '',
       'decimals': 0,
       'logo': 'QmZDWdXXBgJJ5QEvc68xeaJhk3dwJgsVQnSjJnoEtCBRrK',
       'ticker': 'hydrogen',
       'contract': 'hydrogen'}},
     'coins': []}},
   {'type': '/cosmwasm.wasm.v1.MsgExecuteContract',
    'val